In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
torch.cuda.is_available()

False

In [5]:
print(torch.version.cuda)

None


In [5]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))  
        x = self.pool(x)          
        x = nn.functional.relu(self.conv2(x))  
        x = self.pool(x)          
        x = x.reshape(x.shape[0], -1)  
        x = self.fc1(x)       
        return x

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
input_size = 784  # 28x28 pixels (not directly used in CNN)
num_classes = 10  # digits 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [ ]:
train_dataset = datasets.MNIST(root="dataset/", download=False, train=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", download=False, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [02:55<00:00, 56.4kB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 217kB/s]
100%|██████████| 1.65M/1.65M [00:09<00:00, 174kB/s] 
100%|██████████| 4.54k/4.54k [00:00<00:00, 17.1MB/s]


In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Normalize((0.1307,), (0.3081,))  
])

In [9]:
model = CNN(in_channels=1, num_classes=num_classes).to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

Epoch [1/10]


100%|██████████| 938/938 [00:07<00:00, 133.56it/s]


Epoch [2/10]


100%|██████████| 938/938 [00:06<00:00, 138.72it/s]


Epoch [3/10]


...

100%|██████████| 938/938 [00:11<00:00, 78.59it/s] 


Epoch [9/10]


100%|██████████| 938/938 [00:12<00:00, 73.76it/s]


Epoch [10/10]


100%|██████████| 938/938 [00:11<00:00, 79.31it/s]


In [12]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad(): 
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1) 
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")
    
    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59518/60000 with accuracy 99.20%
Checking accuracy on test data
Got 9870/10000 with accuracy 98.70%
